In [1]:
import boto3

In [2]:
g_client = boto3.client('glue',region_name='us-east-1')

In [5]:
tab = g_client.get_table(DatabaseName='copilot_demo',Name='employees_test')

In [12]:
dtypes_check = {}

In [13]:
for col in tab['Table']['StorageDescriptor']['Columns']:
    dtypes_check[col['Name']] = col['Type'] 

In [14]:
dtypes_check

{'emp_id': 'int',
 'name': 'string',
 'salary': 'double',
 'department': 'string',
 'joining_date': 'date'}

In [ ]:
dtype_rules = []
for col,target_type in dtypes_check.items():
    if target_type.lower() == 'string':
        continue
    spark_exp = f'{col} IS NULL OR CAST({col} AS {target_type}) IS NOT NULL'

    dtype_rules.append({
        "column": col,
        "rule_type": "dtype",
        "condition": target_type,
        "severity": "ERROR",
        "action": "FAIL_JOB",
        "description": f"value in {col} must be a valid {target_type}",
        "spark_exp": spark_exp
    })
    

In [26]:
dtype_rules

[{'column': 'emp_id',
  'rule_type': 'dtype',
  'condition': 'int',
  'severity': 'ERROR',
  'action': 'FAIL_JOB',
  'description': 'value in emp_id must be a valid int',
  'spark_exp': 'emp_id is null or cast(emp_id as int) is not null'},
 {'column': 'name',
  'rule_type': 'dtype',
  'condition': 'string',
  'severity': 'ERROR',
  'action': 'FAIL_JOB',
  'description': 'value in name must be a valid string',
  'spark_exp': 'name is null or cast(name as string) is not null'},
 {'column': 'salary',
  'rule_type': 'dtype',
  'condition': 'double',
  'severity': 'ERROR',
  'action': 'FAIL_JOB',
  'description': 'value in salary must be a valid double',
  'spark_exp': 'salary is null or cast(salary as double) is not null'},
 {'column': 'department',
  'rule_type': 'dtype',
  'condition': 'string',
  'severity': 'ERROR',
  'action': 'FAIL_JOB',
  'description': 'value in department must be a valid string',
  'spark_exp': 'department is null or cast(department as string) is not null'},
 {'co